In [1]:
import os
import re
import csv
import pandas as pd

# path to walid's output data

output_path = '/work/AI-EHR/discharge_me/data/Experiments/RAG/Walid_RAG_20240708/hadm_answers_summary_and_letter'

# path to reference data

ref_path = '/work/AI-EHR/discharge_me/data/test_phase_2'

# hadm_id_list

hadm_id_list_df = pd.read_csv(os.path.join(ref_path, 'hadm_ids_for_discharge_me_scoring.csv'))
# hadm_id_list_file = (os.path.join(ref_path, 'hadm_ids_for_discharge_me_scoring.csv'))

# Read the file identifiers from the CSV file
hadm_id_list_df.head()

# list all the txt files with a suffix of _masked_summary_out.txt
summary_out_txt_files = [f for f in os.listdir(output_path) if f.endswith('_masked_summary_out.txt')]
letter_out_txt_files = [f for f in os.listdir(output_path) if f.endswith('_masked_letter_out.txt')]

# refs will be our segmentation of BHC and discharge summaries for each hadm_id in the list
discharge_sections_df = pd.read_csv(os.path.join(ref_path, 'discharge_sections.csv'), keep_default_na=False)

In [2]:
# print(discharge_sections_df['BHC'][0])
discharge_sections_df.head()

,Unnamed: 0,subject_id,hadm_id,CC,Service,Major Surgical Procedure,HPI,PMH,SOC,FH,...,Problem List,Physical Exam,Medication Lists,Pertinent Results,BHC,Transitional Issues,Disposition,Discharge Instructions,Followup Instructions,Discharge Diagnosis
0,0,10001884,24962904,Chief Complaint:\nShortness of Breath,"Service: MEDICINE\n \nAllergies: \nIV Dye, Iod...",Major Surgical or Invasive Procedure:\nN/A,History of Present Illness:\nMs. ___ is a ___ ...,Past Medical History:\n- COPD/Asthma on home 2...,Social History:\n___,Family History:\nMother with asthma and hypert...,...,,Physical Exam:\nADMISSION PHYSICAL EXAM:\n====...,Preadmission Medication list is accurate and c...,Pertinent Results:\nADMISSION LABS: \n========...,Brief Hospital Course:\nMs. ___ is a ___ femal...,TRANSITIONAL ISSUES:\n========================...,Disposition:\nExtended Care\n \nFacility:\n___...,"Discharge Instructions:\nDear Ms. ___,\n\nYou ...",,Diagnosis:\nPRIMARY:\nCOPD Exacerbation\n\nSEC...
1,1,10003019,22774359,Chief Complaint:\nfever,Service: MEDICINE\n \nAllergies: \nRagweed / m...,Major Surgical or Invasive Procedure:\nnone,History of Present Illness:\nMr ___ is a ___ w...,"Past Medical History:\n1. Sarcoidosis, dx skin...",Social History:\n___,"Family History:\nMother: ___, cardiac disease....",...,,Physical Exam:\nADMISSION EXAM\nVitals: 124/67...,Medications on Admission:\nThe Preadmission Me...,Pertinent Results:\nADMISSION LABS\n___ 10:40A...,Brief Hospital Course:\n___ male with h/o Hodg...,,Disposition:\nHome With Service\n \nFacility:\...,"Discharge Instructions:\nDear Mr. ___,\n\nIt h...",,Discharge Diagnosis:\nPrimary Diagnosis\nNeutr...
2,2,10003299,29323205,"Chief Complaint:\nleft leg weakness, falls",Service: NEUROLOGY\n \nAllergies: \nIodine-Iod...,Major Surgical or Invasive Procedure:\nNone,History of Present Illness:\n___ is a ___ RH f...,Past Medical History:\n- prior paramedian pont...,Social History:\n___,Family History:\nMother had stroke in her ___ ...,...,,Physical Exam:\nAdmission Exam:\nVitals: T: 97...,Medications on Admission:\nThe Preadmission Me...,Pertinent Results:\n___ 01:10PM GLUCOSE-125*...,Brief Hospital Course:\n___ RH female with a P...,,Disposition:\nHome With Service\n \nFacility:\...,Discharge Instructions:\nDear ___ were hospita...,,Discharge Diagnosis:\nIschemic stroke
3,3,10003502,20459702,Chief Complaint:\nNausea,Service: MEDICINE\n \nAllergies: \nnifedipine ...,Major Surgical or Invasive Procedure:\nNone,History of Present Illness:\nMs. ___ is an ___...,Past Medical History:\nHypertension/hyperlipid...,Social History:\n___,Family History:\nMother deceased at ___ yo fro...,...,,Physical Exam:\nOn admission: \nVS 97.4 140/P ...,Medications on Admission:\nThe Preadmission Me...,Pertinent Results:\n___ 10:15AM BLOOD WBC-6.4 ...,Brief Hospital Course:\nHospitalization Summar...,Transitional Issues: \n- monitoring of volume ...,Disposition:\nHome With Service\n \nFacility:\...,Discharge Instructions:\nIt was a pleasure car...,,Discharge Diagnosis:\nPrimary: \nAcute diastol...
4,4,10004322,28755331,Chief Complaint:\nmultiple falls at group home,Service: MEDICINE\n \nAllergies: \nNo Known Al...,Major Surgical or Invasive Procedure:\nNone,History of Present Illness:\nMr. ___ is a ___ ...,Past Medical History:\nPsychosis \nDiabetes \n...,Social History:\n___,Family History:\nUnknown to patient.,...,,Physical Exam:\nADMISSION PHYSICAL EXAM: \nVS:...,Medications on Admission:\nThe Preadmission Me...,Pertinent Results:\nADMISSION LABS:\n___ 03:40...,Brief Hospital Course:\n___ with h/o psychosis...,TRANSITIONAL ISSUES:\n-He was discharged back ...,Disposition:\nHome With Service\n \nFacility:\...,"Discharge Instructions:\nDear Mr. ___,\n\nIt w...",,Discharge Diagnosis:\nPrimary: mechanical fall...


In [29]:
summary_out_txt_files?

Type:        list
String form: ['hadm_id_20226692_inputs_summary_out.txt_masked_summary_out.txt', 'hadm_id_20279012_inputs_summa <...> ut.txt_masked_summary_out.txt', 'hadm_id_29991915_inputs_summary_out.txt_masked_summary_out.txt']
Length:      243
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


In [16]:
## extract text data from walid's text files and put them in a csv/data frame with the corresponding human generated target data

file_list = summary_out_txt_files #for BHC LLM outputs

# Function to extract numeric identifier from filename
def extract_identifier(filename):
  
    match = re.search(r'_(\d+)_', filename)
    if match:
        return match.group(1)
    else:
        return None

# Initialize a list to store the data
data = []

# Iterate through each file in the list
for file_name in file_list:
    filename = os.path.join(output_path, file_name)
    identifier = extract_identifier(file_name)
    if identifier:
        with open(filename, 'r') as file:
            content = file.read()
            data.append({'identifier': identifier, 'content': content})

# Create a pandas DataFrame
df = pd.DataFrame(data)

df.rename(columns={'identifier': 'hadm_id', 'content': 'BHC_LLM_out'}, inplace=True)
df['hadm_id'] = df['hadm_id'].astype(int)

# Merge with target data, in this case human generated sections

merged_df = df.merge(discharge_sections_df[['hadm_id', 'BHC']], on='hadm_id', how='left')
merged_df.rename(columns={'BHC': 'BHC_human_generated'}, inplace=True)

# Optionally, save the DataFrame to a CSV file
merged_df.to_csv(os.path.join(output_path, 'summary_out_with_targets.csv'), index=False)
summary_out_df = merged_df

##  for discharge instruction letter outputs
file_list = letter_out_txt_files #for discharge instructions letter outputs

# Initialize a list to store the data
data = []

# Iterate through each file in the list
for file_name in file_list:
    filename = os.path.join(output_path, file_name)
    identifier = extract_identifier(file_name)
    if identifier:
        with open(filename, 'r') as file:
            content = file.read()
            data.append({'identifier': identifier, 'content': content})

# Create a pandas DataFrame
df = pd.DataFrame(data)

df.rename(columns={'identifier': 'hadm_id', 'content': 'Letter_LLM_out'}, inplace=True)
df['hadm_id'] = df['hadm_id'].astype(int)

# Merge with target data, in this case human generated sections

merged_df = df.merge(discharge_sections_df[['hadm_id', 'Discharge Instructions']], on='hadm_id', how='left')
merged_df.rename(columns={'Discharge Instructions': 'Letter_human_generated'}, inplace=True)
                          
# merged_df.head(3)


# # Optionally, save the DataFrame to a CSV file
merged_df.to_csv(os.path.join(output_path, 'letter_out_with_targets.csv'), index=False)
letter_out_df = merged_df


In [17]:
letter_out_df

,hadm_id,Letter_LLM_out,Letter_human_generated
0,20202032,"Dear Patient,\n\nI hope this letter finds you ...","Discharge Instructions:\nDear Mr. ___,\n\nIt w..."
1,20226692,"Dear Patient,\n\nI hope this letter finds you ...","Discharge Instructions:\nMs. ___,\n\nYou were ..."
2,20279012,"Dear Patient,\n\nI hope this letter finds you ...","Discharge Instructions:\nDear Mr. ___, \n\nIt..."
3,20289285,"Dear [Patient's Name],\n\nI hope this letter f...","Discharge Instructions:\nDear Mr. ___,\n\nYou ..."
4,20390086,We regret to inform you that we are currently ...,Discharge Instructions:\nPlease call Dr. ___ o...
...,...,...,...
239,29968521,"Dear Patient,\n\nI hope this letter finds you ...","Discharge Instructions:\nDear ___,\n\n___ were..."
240,29980084,"Dear Patient,\n\nI hope this letter finds you ...","Discharge Instructions:\nDear Dr ___,\n\n___ w..."
241,29984324,"Dear Patient,\n\nI hope this letter finds you ...","Discharge Instructions:\nDear Mr. ___, \nYou w..."
242,29989258,"Dear [Patient's Name],\n\nI hope this letter f...","Discharge Instructions:\nDear Mr. ___,\n\nIt w..."


In [5]:
print(letter_out_df['Letter_LLM_out'][0])

Dear Patient,

I hope this letter finds you well. I wanted to take a moment to provide you with an update on your recent hospital admission.

You were admitted to the hospital due to concerns regarding your heart health. The main reason for your admission was to investigate chest pain and shortness of breath that you had been experiencing. After conducting various tests, it was determined that you had a blockage in one of your coronary arteries, which was affecting the blood flow to your heart muscle.

During your stay, you underwent several diagnostic tests, including an electrocardiogram (ECG) and a cardiac catheterization. These tests helped us confirm the blockage in your artery and assess the extent of the damage to your heart.

Based on the results of these tests, you received a coronary angioplasty and stent placement procedure to open up the blocked artery and restore proper blood flow to your heart. This treatment was successful, and you responded well to the procedure.

Curre

## letter scoring

In [ ]:
!pip install evaluate nltk bert_score

In [14]:
def preprocess_text(text, tokenize=False):
    # Expand medical acronyms
    # text = expand_acronyms(text) #TODO
    
    # Remove HTML/XML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove non-alphanumeric characters except ., ? : = )
    text = re.sub(r'[^a-zA-Z0-9\s\.\:\?\=]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    if tokenize:
        # Tokenize the text
        tokens = word_tokenize(text)
        
        # Remove stop words #TODO
        # stop_words = set(stopwords.words('english'))
        # tokens = [token for token in tokens if token not in stop_words]
        
        return ' '.join(tokens)
    else:
        return text

In [18]:
processed_letters_txt = pd.DataFrame()
processed_letters_txt['hadm_id'] = letter_out_df['hadm_id']
processed_letters_txt['Letter_LLM_out'] = ''
processed_letters_txt['Letter_human_generated'] = ''

for i in range(len(letter_out_df)):
    processed_letters_txt.loc[i, 'Letter_LLM_out'] = preprocess_text(letter_out_df.loc[i, 'Letter_LLM_out'])
    processed_letters_txt.loc[i, 'Letter_human_generated'] = preprocess_text(letter_out_df.loc[i, 'Letter_human_generated'])

processed_letters_txt

,hadm_id,Letter_LLM_out,Letter_human_generated
0,20202032,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear mr. \n\nit was a...
1,20226692,dear patient\n\ni hope this letter finds you w...,discharge instructions:\nms. \n\nyou were admi...
2,20279012,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear mr. \n\nit was...
3,20289285,dear patients name\n\ni hope this letter finds...,discharge instructions:\ndear mr. \n\nyou were...
4,20390086,we regret to inform you that we are currently ...,discharge instructions:\nplease call dr. offi...
...,...,...,...
239,29968521,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear \n\n were admitt...
240,29980084,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear dr \n\n were adm...
241,29984324,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear mr. \nyou were ...
242,29989258,dear patients name\n\ni hope this letter finds...,discharge instructions:\ndear mr. \n\nit was a...


In [19]:
from evaluate import load

meteor = load('meteor')
scores_df = pd.DataFrame()
bertscore = load("bertscore")

# Copy relevant columns from the original DataFrame
scores_df['hadm_id'] = processed_letters_txt['hadm_id']
scores_df['Letter_LLM_out'] = processed_letters_txt['Letter_LLM_out']
scores_df['Letter_human_generated'] = processed_letters_txt['Letter_human_generated']
scores_df['bertscore_precision'] = 0.0
scores_df['bertscore_recall'] = 0.0
scores_df['bertscore_f1'] = 0.0

# Initialize the 'meteor_score' column with zeros
scores_df['meteor_score'] = 0.0

# Calculate METEOR score for each row
for index, row in scores_df.iterrows():
    prediction = [row['Letter_LLM_out']]
    reference = [row['Letter_human_generated']]
    result = meteor.compute(predictions=prediction, references=reference)
    scores_df.at[index, 'meteor_score'] = result['meteor']
    bertscore_result = bertscore.compute(predictions=prediction, references=reference, model_type="distilbert-base-uncased")
    scores_df.at[index, 'bertscore_precision'] = bertscore_result['precision'][0]
    scores_df.at[index, 'bertscore_recall'] = bertscore_result['recall'][0]
    scores_df.at[index, 'bertscore_f1'] = bertscore_result['f1'][0]
    
scores_df

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mahajan.d/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/mahajan.d/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mahajan.d/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,hadm_id,Letter_LLM_out,Letter_human_generated,bertscore_precision,bertscore_recall,bertscore_f1,meteor_score
0,20202032,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear mr. \n\nit was a...,0.746939,0.667188,0.704814,0.260308
1,20226692,dear patient\n\ni hope this letter finds you w...,discharge instructions:\nms. \n\nyou were admi...,0.761101,0.765093,0.763092,0.253837
2,20279012,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear mr. \n\nit was...,0.752073,0.683631,0.716221,0.245312
3,20289285,dear patients name\n\ni hope this letter finds...,discharge instructions:\ndear mr. \n\nyou were...,0.765008,0.793914,0.779193,0.257218
4,20390086,we regret to inform you that we are currently ...,discharge instructions:\nplease call dr. offi...,0.735952,0.740524,0.738231,0.195581
...,...,...,...,...,...,...,...
239,29968521,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear \n\n were admitt...,0.775582,0.784258,0.779896,0.288755
240,29980084,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear dr \n\n were adm...,0.715304,0.764968,0.739303,0.177552
241,29984324,dear patient\n\ni hope this letter finds you w...,discharge instructions:\ndear mr. \nyou were ...,0.767597,0.768171,0.767884,0.257900
242,29989258,dear patients name\n\ni hope this letter finds...,discharge instructions:\ndear mr. \n\nit was a...,0.769866,0.807568,0.788266,0.318944


In [20]:
scores_df.describe()

,hadm_id,bertscore_precision,bertscore_recall,bertscore_f1,meteor_score
count,2.440000e+02,244.000000,244.000000,244.000000,244.000000
mean,2.509075e+07,0.751922,0.781819,0.766283,0.255358
std,2.674783e+06,0.017392,0.032644,0.021554,0.042787
min,2.020203e+07,0.632377,0.621704,0.626995,0.042888
25%,2.286709e+07,0.740744,0.770856,0.758744,0.227632
50%,2.490527e+07,0.753964,0.786863,0.769505,0.256461
75%,2.719655e+07,0.763141,0.802115,0.779169,0.281263
max,2.999192e+07,0.792266,0.833679,0.806796,0.380037


In [21]:
scores_df.to_csv("letter_scores_output.csv",index=False)

## summary scoring

In [5]:
%pwd

'/work/AI-EHR/discharge_me/Notebooks/Dushyant_notebooks'

In [9]:
summary_df = pd.read_csv("/work/AI-EHR/discharge_me/Notebooks/Dushyant_notebooks/data/summary_out_with_targets.csv")
summary_df

,hadm_id,BHC_LLM_out,BHC_human_generated
0,20226692,The patient was admitted to the hospital due t...,"Brief Hospital Course:\nOn ___, Ms. ___ was ad..."
1,20279012,The patient was admitted to the hospital due t...,Brief Hospital Course:\nHospital course summar...
2,20289285,The patient was admitted to the cardiology ser...,Brief Hospital Course:\n___ PMH severe dilated...
3,20390086,The patient was admitted to the hospital due t...,Brief Hospital Course:\nP: ___ y/o F with uncl...
4,20449726,The patient was admitted to the hospital for s...,Brief Hospital Course:\nMr. ___ was admitted t...
...,...,...,...
238,29968521,"The patient has a history of dyslipidemia, hyp...",Brief Hospital Course:\n___ with PMH including...
239,29980084,"The patient has a history of hypertension, LVH...",Brief Hospital Course:\n___ year old male p/w ...
240,29984324,The patient was admitted to the hospital due t...,Brief Hospital Course:\nMr. ___ is a ___ old m...
241,29989258,The patient was admitted to the Medicine servi...,Brief Hospital Course:\n___ w/ PMH of SVT with...


In [12]:
from evaluate import load

meteor = load('meteor')
bertscore = load("bertscore")
summary_scores_df = pd.DataFrame()

# Copy relevant columns from the original DataFrame
summary_scores_df['hadm_id'] = summary_df['hadm_id']
summary_scores_df['BHC_LLM_out'] = summary_df['BHC_LLM_out']
summary_scores_df['BHC_human_generated'] = summary_df['BHC_human_generated']

# Initialize the 'meteor_score' column with zeros
summary_scores_df['meteor_score'] = 0.0
summary_scores_df['bertscore_precision'] = 0.0
summary_scores_df['bertscore_recall'] = 0.0
summary_scores_df['bertscore_f1'] = 0.0

# Calculate METEOR score for each row
for index, row in summary_scores_df.iterrows():
    prediction = [row['BHC_LLM_out']]
    reference = [row['BHC_human_generated']]
    result = meteor.compute(predictions=prediction, references=reference)
    summary_scores_df.at[index, 'meteor_score'] = result['meteor']
    bertscore_result = bertscore.compute(predictions=prediction, references=reference, model_type="distilbert-base-uncased")
    summary_scores_df.at[index, 'bertscore_precision'] = bertscore_result['precision'][0]
    summary_scores_df.at[index, 'bertscore_recall'] = bertscore_result['recall'][0]
    summary_scores_df.at[index, 'bertscore_f1'] = bertscore_result['f1'][0]

summary_scores_df

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mahajan.d/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/mahajan.d/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mahajan.d/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,hadm_id,BHC_LLM_out,BHC_human_generated,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1
0,20226692,The patient was admitted to the hospital due t...,"Brief Hospital Course:\nOn ___, Ms. ___ was ad...",0.125196,0.772472,0.757634,0.764981
1,20279012,The patient was admitted to the hospital due t...,Brief Hospital Course:\nHospital course summar...,0.095847,0.804317,0.726384,0.763366
2,20289285,The patient was admitted to the cardiology ser...,Brief Hospital Course:\n___ PMH severe dilated...,0.112045,0.778857,0.742765,0.760383
3,20390086,The patient was admitted to the hospital due t...,Brief Hospital Course:\nP: ___ y/o F with uncl...,0.131492,0.780739,0.722548,0.750518
4,20449726,The patient was admitted to the hospital for s...,Brief Hospital Course:\nMr. ___ was admitted t...,0.212954,0.799560,0.799988,0.799774
...,...,...,...,...,...,...,...
238,29968521,"The patient has a history of dyslipidemia, hyp...",Brief Hospital Course:\n___ with PMH including...,0.145128,0.771547,0.763387,0.767446
239,29980084,"The patient has a history of hypertension, LVH...",Brief Hospital Course:\n___ year old male p/w ...,0.126582,0.751548,0.714614,0.732616
240,29984324,The patient was admitted to the hospital due t...,Brief Hospital Course:\nMr. ___ is a ___ old m...,0.148472,0.768029,0.741577,0.754571
241,29989258,The patient was admitted to the Medicine servi...,Brief Hospital Course:\n___ w/ PMH of SVT with...,0.151278,0.802587,0.687515,0.740608


In [13]:
summary_scores_df.to_csv("summary_scores_output.csv",index=False)

In [60]:
import pandas as pd
import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import mauve
import bert_score
import sacrebleu
from rouge_score import rouge_scorer
# import moverscore_v2 as mv_sc
import re
import torch 



# Download required NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Example custom medical acronym dictionary #TODO
medical_acronyms = {
    'MI': 'myocardial infarction',
    'COPD': 'chronic obstructive pulmonary disease',
    'DM': 'diabetes mellitus',
    'CHF': 'congestive heart failure',
    'HTN': 'hypertension',
    'CAD': 'coronary artery disease',
    # Add more acronyms as needed
}

# Specific Metric Requirements:
# METEOR: Requires tokenized input but does not require padding.
# GLEU: Typically uses tokenized sentences, no padding required.
# BERTScore: Handles tokenization internally.
# Rouge-L: Handles tokenization internally.
# MoverScore: Requires tokenized input, no padding required.
# Mauve: Handles tokenization internally.

# Function to remove extraneous text and punctuation, with tokenization flag
def preprocess_text(text, tokenize=False):
    # Expand medical acronyms
    # text = expand_acronyms(text) #TODO
    
    # Remove HTML/XML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove non-alphanumeric characters except ., ? : = )
    text = re.sub(r'[^a-zA-Z0-9\s\.\:\?\=]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    if tokenize:
        # Tokenize the text
        tokens = word_tokenize(text)
        
        # Remove stop words #TODO
        # stop_words = set(stopwords.words('english'))
        # tokens = [token for token in tokens if token not in stop_words]
        
        return ' '.join(tokens)
    else:
        return text


# Function to expand medical acronyms #TODO
def expand_acronyms(text):
    for acronym, expansion in medical_acronyms.items():
        text = re.sub(r'\b{}\b'.format(acronym), expansion, text)
    return text

# Custom stemmer that handles medical acronyms
class MedicalStemmer:
    def __init__(self):
        self.porter = PorterStemmer()
    
    def stem(self, word):
        return self.porter.stem(word)
    
    def stem_text(self, text):
        expanded_text = expand_acronyms(text)
        tokens = word_tokenize(expanded_text.lower())
        stemmed_tokens = [self.stem(token) for token in tokens]
        return ' '.join(stemmed_tokens)
    
    

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    # METEOR: Requires tokenized input but does not require padding.
    # return sacrebleu.metrics.meteor.METEOR().score([hypothesis], [[reference]])
    return meteor_score([reference], hypothesis)

# Function to calculate GLEU score
def calculate_gleu(hypothesis, reference):
    # GLEU: Typically uses tokenized sentences, no padding required.
    return sacrebleu.sentence_gleu([reference], hypothesis)

# Function to calculate BERTScore using BioBERT
def calculate_bertscore(hypothesis, reference):
    # BERTScore: Handles tokenization internally.

    P, R, F1 = bert_score.score(
        [hypothesis], [reference], 
        model_type='monologg/biobert_v1.1_pubmed', 
        lang='en', 
        rescale_with_baseline=True
    )
    return P.mean().item(), R.mean().item(), F1.mean().item()

# Function to calculate Rouge-L score
def calculate_rouge_l(hypothesis, reference):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores['rougeL'].fmeasure

# Function to calculate MoverScore
def calculate_moverscore(hypothesis, reference): #TODO
    # return mv_sc.get_scores([hypothesis], [reference], n_gram=1)[0]
    return 

# Function to calculate Mauve score
def calculate_mauve(hypothesis, reference):
    mauve_result = mauve.compute_mauve(p_text=[hypothesis], q_text=[reference])
    return mauve_result.mauve

# Function to calculate and append scores to the DataFrame
def append_scores(df, hyp_col, ref_col):
    scores = {
        'METEOR': [], 'GLEU': [], 'BERTScore_P': [], 'BERTScore_R': [],
        'BERTScore_F1': [], 'Rouge-L': [], 'MoverScore': [], 'Mauve': []
    } 
    for idx, row in df.iterrows():
        hyp_raw = row[hyp_col]
        ref_raw = row[ref_col]
        
      # Preprocessed text for tokenization-dependent metrics
        hyp_tokenized = preprocess_text(hyp_raw, tokenize=True)
        ref_tokenized = preprocess_text(ref_raw, tokenize=True)
        
        hyp_raw = preprocess_text(hyp_raw, tokenize=False)
        ref_raw = preprocess_text(ref_raw, tokenize=False)

        # Calculate scores
        scores['METEOR'].append(calculate_meteor(hyp_tokenized, ref_tokenized))
        scores['GLEU'].append(calculate_gleu(hyp_tokenized, ref_tokenized))
        P, R, F1 = calculate_bertscore(hyp_raw, ref_raw)
        scores['BERTScore_P'].append(P)
        scores['BERTScore_R'].append(R)
        scores['BERTScore_F1'].append(F1)
        scores['Rouge-L'].append(calculate_rouge_l(hyp_raw, ref_raw))
        # scores['MoverScore'].append(calculate_moverscore(hyp_tokenized, ref_tokenized)) #todo
        # scores['Mauve'].append(calculate_mauve(hyp_raw, ref_raw))
   
    for metric in scores:
        df[metric] = scores[metric]
    
    return df

# Call the function with the merged_df, hypothesis column, and reference column
summary_result_df = append_scores(summary_out_df, 'BHC_LLM_out', 'BHC_human_generated')

# Display the DataFrame with scores
print(summary_result_df)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/s.wendelken/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/s.wendelken/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/s.wendelken/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


TypeError: "hypothesis" expects pre-tokenized hypothesis (Iterable[str]): the patient was admitted to the hospital due to exacerbation of congestive heart failure . they were admitted to the cardiology service and underwent a cardiac catheterization procedure . for the congestive heart failure the patient presented with shortness of breath and lower extremity edema . the differential diagnosis included acute decompensated heart failure and pneumonia . diagnostic workup included chest xray showing pulmonary congestion bnp levels were elevated and echocardiogram revealed reduced ejection fraction . treatment involved diuresis with iv furosemide and ace inhibitor therapy . physical exam findings included bilateral crackles in the lungs and jugular venous distension . lab work showed elevated bnp levels . pending results include renal function tests . followup includes outpatient cardiology visit in 2 weeks . medications administered were iv furosemide and lisinopril .

In [38]:
summary_out_df.head()

,hadm_id,BHC_LLM_out,BHC_human_generated
0,20226692,The patient was admitted to the hospital due t...,"Brief Hospital Course:\nOn ___, Ms. ___ was ad..."
1,20279012,The patient was admitted to the hospital due t...,Brief Hospital Course:\nHospital course summar...
2,20289285,The patient was admitted to the cardiology ser...,Brief Hospital Course:\n___ PMH severe dilated...
3,20390086,The patient was admitted to the hospital due t...,Brief Hospital Course:\nP: ___ y/o F with uncl...
4,20449726,The patient was admitted to the hospital for s...,Brief Hospital Course:\nMr. ___ was admitted t...


In [53]:
# !pip install nltk mauve-text sacrebleu rouge-score evaluate
# !pip install bert-score torch
# !pip install moverscore
# !pip install pyemd
# !pip install transformers
!pip install pytest

Defaulting to user installation because normal site-packages is not writeable
